# Reinforcement Learning Tasks

En el problemas de aprendizaje, a diferencia de en programación dinámica, no conocemos los detalles del problema. Por lo tanto vamos a modelar un problema de aprendizaje con dos "clases". 

1. La primera clase va a definir la tarea. Vamos a nombrar a esta tarea RLTask
2. La segunda clase define al agente que aprende a resolver la tarea maximizando su ganancia.

Vamos a ver como construir cada una de estas clases de manera que tengamos un cascarón para distintas tareas de aprendizaje.